# Mamba-2 Language Model demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time

import torch
from transformers import AutoTokenizer

from mamba2 import Mamba2LMHeadModel

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

Official pretrained models on [huggingface](https://huggingface.co/state-spaces):
* `state-spaces/mamba2-130m`
* `state-spaces/mamba2-370m`
* `state-spaces/mamba2-780m`
* `state-spaces/mamba2-1.3b`
* `state-spaces/mamba2-2.7b`

Note that these are base models without fine-tuning for downstream tasks such as chat or instruction following.

In [3]:
# Choose a model depending on available system RAM (for Apple Silicon) or VRAM.
model = Mamba2LMHeadModel.from_pretrained("state-spaces/mamba2-1.3b", device=device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
generation_config = dict(
    max_new_length=200,
    temperature=1.0,
    top_k=30,
    top_p=1.0,
)

In [5]:
def generate(prompt: str, seed: int = 0, show_perf: bool = True):
    """Generate streaming completion"""
    torch.manual_seed(seed)

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)[0]
    print(prompt, end="")

    start = time.process_time()
    n_generated = 0
    for i, (token_id, _hidden_state) in enumerate(model.generate(input_ids, **generation_config)):
        token = tokenizer.decode([token_id])
        if i == 0:
            now = time.process_time()
            prompt_eval_elapsed, start = now - start, now
        else:
            n_generated += 1
        print(token, end="", flush=True)
    if show_perf:
        elapsed = time.process_time() - start
        print('\n\n---')
        print(f'Prompt eval | tokens: {input_ids.shape[0]} | elapsed: {prompt_eval_elapsed:.2f}s | tok/s: {input_ids.shape[0] / prompt_eval_elapsed:.2f}')
        print(f'Generation | tokens: {n_generated} | elapsed: {elapsed:.2f}s | tok/s: {n_generated / elapsed:.2f}')

In [6]:
generate("Mamba is a new state space model architecture")

Mamba is a new state space model architecture that enables the modeling of discrete events
using the M-ary BDD model

The Mamba architecture
defines a state space model using a novel state abstraction that enables
state to change and transition to a new state based on the values that change
in any M-ary BDD node.

Mamba’s state abstraction provides a unified representation of an
event’s effects and effects of past events. When the M-ary BDD nodes
change, each one of their corresponding states is represented with an
indicator that changes when any state is changed from any other node to
maintain a trace of changes. Mamba enables the modeling of
events that can occur both synchronously or asynchronously.

State transition is based on three types of
events:

Transition based on the change of any of the nodes of a M-ary BDD

Transition based on an event being fired based on the change
of any of the nodes

---
Prompt eval | tokens: 9 | elapsed: 1.41s | tok/s: 6.38
Generation | tokens: 199 |

In [7]:
generate("The meaning of life is")

The meaning of life is death

The meaning of life is that people die and the living have to carry on

The meaning of life is that if we don't do things today we have to do them tomorrow

---
Prompt eval | tokens: 5 | elapsed: 0.75s | tok/s: 6.66
Generation | tokens: 38 | elapsed: 2.55s | tok/s: 14.90


In [8]:
generate("CUDA is Nvidia's biggest moat")

CUDA is Nvidia's biggest moat on NVidia's biggest moat

"While CUDA is Nvidia's biggest moat for the time being, it's also clear that Nvidia has a huge opportunity to improve the performance and capabilities of its GPUs to keep pace with a new generation of processors based on ARM's big.LITTLE architecture."

The problem with CUDA is that even if your game runs well on one of their products, the hardware is still far inferior to the competition. I can buy a Nvidia card that has better performance, but I can't do much to improve its design.

When it comes to performance I'll stick to the Radeon 7990.

I can also buy a GPU card, but I can't really do much to improve its capability other than buy more memory. While this might sound like a weak point for Nvidia, I would say that it's a strength for AMD, the market is there for both of them. If AMD wanted,

---
Prompt eval | tokens: 9 | elapsed: 0.73s | tok/s: 12.27
Generation | tokens: 199 | elapsed: 12.72s | tok/s: 15.64
